 环境准备
- pyenv安装 
```shell
brew update
brew install pyenv
# 设置环境变亮
echo -e 'export PATH="$HOME/.pyenv/bin:$PATH"\neval "$(pyenv init --path)"\neval "$(pyenv virtualenv-init -)"' >> ~/.bashrc
# 重启
exec "$SHELL"
```
- 创建隔离虚拟环境
```shell
pyenv install 3.12.2
pyenv virtualenv 3.12.2 langchain
pyenv local langchain
```

#### 测试环境初运行

In [2]:
print('hello world')

hello world


#### 安装langchain

In [3]:
! pip install --upgrade langchain==0.0.279 -i https://pypi.org/simple

  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.52
    Uninstalling langsmith-0.1.52:
      Successfully uninstalled langsmith-0.1.52
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.1.48 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.


### 原生使用openai

#### 安装openai库

In [4]:
! pip install openai

#### 设置环境变量key

In [3]:
import os

os.environ["OPENAI_KEY"] = "sk-xxx"
# os.environ["OPENAI_API_BASE"] = "https://solkatt.vip/proxy/https://api.openai.com/v1"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"

#### 读取环境变量

In [2]:
! pip show langchain
! pip show openai

Name: langchain
Version: 0.0.279
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/barry/.pyenv/versions/3.12.2/envs/langchain/lib/python3.12/site-packages
Requires: aiohttp, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Name: openai
Version: 1.25.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /Users/barry/.pyenv/versions/3.12.2/envs/langchain/lib/python3.12/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai


In [4]:
import os
openai_api_key = os.getenv("OPENAI_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")
print("OPENAI_API_KEY:", openai_api_key)
print("OPENAI_PROXY:", openai_api_base)

OPENAI_API_KEY: sk-xxx
OPENAI_PROXY: https://api.chatanywhere.tech/v1


#### 调用官方SDK

In [31]:
#使用openai的官方sdk
from openai import OpenAI
import os

# openai.api_base = os.getenv("OPENAI_API_BASE")
# openai.api_key = os.getenv("OPENAI_KEY")

client = OpenAI(
  api_key=os.getenv("OPENAI_KEY"),
  base_url=os.getenv("OPENAI_API_BASE")
)

completion = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


### 使用langchain调用

#### 安装必要的库

In [ ]:
! pip install langchain_core
! pip install langchain_openai

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

llm = OpenAI(
  openai_api_key=os.getenv("OPENAI_KEY"), 
  # openai_api_base=os.getenv("OPENAI_API_BASE"),
  temperature=0,
  # model_name="gpt-4"
)

question = "什么是NFT呢?"

llm_chain = prompt | llm
llm_chain.invoke(question)

' \n\n首先，NFT是Non-Fungible Token的缩写，中文翻译为“非同质化代币”。它是一种基于区块链技术的数字资产，每一个NFT都是独一无二的，不可替代的。与传统的加密货币不同，NFT不具备可互换性，每一个NFT都有自己独特的属性和价值。\n\n其次，NFT可以代表任何形式的数字资产，比如艺术品、音乐、视频、游戏道具等。它们都可以被转换成NFT，并且在区块链上进行唯一的标识和交易。\n\n最后，NFT的价值和稀缺性有关。由于每一个NFT都是独一无二的，它们的价值也会因为稀缺性而上升。比如一幅艺术作品的NFT，'

#### 完整的调用流程

- 起名服务

In [63]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.exceptions import OutputParserException

class BaseOutputParser(BaseOutputParser[bool]):
    """Custom boolean parser."""

    true_val: str = "YES"
    false_val: str = "NO"

    def parse(self, text: str) -> bool:
        return text.strip().split(",")

    @property
    def _type(self) -> str:
        return "boolean_output_parser"

prompt_template = PromptTemplate.from_template(
    "你是一个起名大师,请模仿示例起3个具有{county}特色的名字。然后只返回以逗号分隔的文本，不要返回其他内容。"
)
message = prompt_template.format(county="中国")

llm = OpenAI(
  openai_api_key=os.getenv("OPENAI_KEY"), 
  # openai_api_base=os.getenv("OPENAI_API_BASE"),
  # temperature=0,
  # model_name="gpt-4"
)
print('template: ', message)
strs = llm.invoke(message)
print('returs strs: ', strs)
format_res = BaseOutputParser().parse(strs)
print('format strs: ', format_res)


template:  你是一个起名大师,请模仿示例起3个具有中国特色的名字。然后只返回以逗号分隔的文本，不要返回其他内容。
returs strs:  

李明, 张晓梅, 王小华
format strs:  ['李明', ' 张晓梅', ' 王小华']
